In [1]:
import nltk
import re
import os
import csv
import pandas as pd
import numpy as np
import pickle

In [2]:
def load_file(filename):
    data = []
    file = open(filename, "r")
    for line in file:
        data.append(line)
    return data

In [3]:
pos_tweets = pd.DataFrame(load_file('twitter-datasets/train_pos_full.txt'), columns=['tweet'])
pos_tweets['sentiment'] = 1

In [4]:
neg_tweets = pd.DataFrame(load_file('twitter-datasets/train_neg_full.txt'), columns=['tweet'])
neg_tweets['sentiment'] = -1

In [5]:
train_tweets = pd.concat([pos_tweets, neg_tweets], axis=0)

In [6]:
test_tweets = pd.DataFrame(load_file('twitter-datasets/test_data.txt'), columns=['tweet'])
test_tweets['tweet'] = test_tweets['tweet'].apply(lambda tweet: tweet.split(',', 1)[-1])

In [7]:
from preprocessing import tweets_preprocessing

In [9]:
train_tweets = tweets_preprocessing(train_tweets)

Contraction words expansion finished!
Emoji to hashtag transformation finished!
Sentiment words emphasizing finished!
Number to hashtag <number> transformation finished!
ALL DONE!


In [8]:
test_tweets = tweets_preprocessing(test_tweets)

Contraction words expansion finished!
Emoji to hashtag transformation finished!
Sentiment words emphasizing finished!
Number to hashtag <number> transformation finished!
ALL DONE!


In [2]:
train_tweets = pd.read_pickle('train_tweets_after_preprocess.pkl')
test_tweets = pd.read_pickle('test_tweets_after_preprocess.pkl')

In [3]:
from sklearn.feature_extraction.text import TfidfVectorizer

def load_vectorizer(train_tweets, test_tweets):
    tfidf = TfidfVectorizer(min_df = 1, max_df = 1, sublinear_tf = True, use_idf = True, ngram_range = (1,4))
    
    train_reptweets = tfidf.fit_transform(train_tweets['tweet'])
    f = open('train_reptweets.pkl','wb')
    # Save dictionary train_reptweets into pickle file f
    pickle.dump(train_reptweets, f)
    
    test_reptweets = tfidf.transform(test_tweets['tweet'])
    f = open('test_reptweets.pkl','wb')
    pickle.dump(test_reptweets, f)
    return train_reptweets, test_reptweets

In [4]:
train_reptweets, test_reptweets = load_vectorizer(train_tweets, test_tweets)

KeyboardInterrupt: 

In [ ]:
from sklearn import svm

# Initialization
clf = svm.LinearSVC(max_iter=10000,intercept_scaling=1,loss='squared_hinge')

# Model training
clf.fit(train_reptweets, train_tweets['sentiment'])

# Prediction
pred = clf.predict(test_reptweets)

In [ ]:
def create_csv_file(results,filepath):
    with open(filepath, 'w') as file:
        fieldnames = ['Id', 'Prediction']
        writeFile = csv.DictWriter(file, delimiter=",", fieldnames=fieldnames)
        writeFile.writeheader()
        id_ = 1
        for result in results:
            writeFile.writerow({'Id':int(id_),'Prediction':result})
            id_ += 1

In [ ]:
# Generating csv file
create_csv_file(pred,'svm_submission.csv')